<a href="https://colab.research.google.com/github/USMANKHALID-GH/MachineLearning/blob/master/Self_supervised_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

Data loading

In [14]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.cifar10.load_data()

In [20]:
print(f"y_train before categorical : {y_train.shape}"); print(f"y_test before categorical: {y_test.shape}")

y_train before categorical : (50000, 1)
y_test before categorical: (10000, 1)


In [21]:
x_train=x_train/255
x_test=x_test/255

In [22]:
y_test=tf.keras.utils.to_categorical(y_test)
y_train=tf.keras.utils.to_categorical(y_train)

In [23]:
y_test[:10]

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [24]:
print(f"y_train after categorical : {y_train.shape}"); print(f"y_test after categorical: {y_test.shape}")
print(f"x_train  : {x_train.shape}"); print(f"x_test : {x_test.shape}")

y_train after categorical : (50000, 10)
y_test after categorical: (10000, 10)
x_train  : (50000, 32, 32, 3)
x_test : (10000, 32, 32, 3)


creating constrative input and output for our data

In [25]:
fun_augment=tf.keras.layers.RandomRotation(factor=0.2)
x_train_augmented=fun_augment(x_train)
print(f"augmented dataset shape: {x_train_augmented.shape}")
#  concat the original dataset and augmented dataset for the pretext work
x_train_pretext=tf.concat([x_train,x_train_augmented],axis=0)

print(f"combined dataset for pretext shape: {x_train_pretext.shape}")

augmented dataset shape: (50000, 32, 32, 3)
combined dataset for pretext shape: (100000, 32, 32, 3)


In [28]:
# output
print(f" x_ train first layers : {x_train.shape[0]}")
print(f" x_ train first layers : {x_train_augmented.shape[0]}")
data_output_pretext_positive=tf.ones((x_train.shape[0],1))
data_output_pretext_negative=tf.zeros((x_train_augmented.shape[0],1))

 x_ train first layers : 50000
 x_ train first layers : 50000


In [32]:
x_train_pretext_output=tf.concat([data_output_pretext_positive,data_output_pretext_negative],axis=0)
print(f"pretext dataset output: {x_train_pretext_output.shape}")
print(f"dataset example: {x_train_pretext_output[:10]}")

pretext dataset output: (100000, 1)
dataset example: [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]


In [31]:
x_train_pretext_output=tf.keras.utils.to_categorical(x_train_pretext_output)
print(f"pretext dataset output: {x_train_pretext_output.shape}")
print(f"dataset example: {x_train_pretext_output[:10]}")

pretext dataset output: (100000, 2, 2)
dataset example: [[[1. 0.]
  [0. 1.]]

 [[1. 0.]
  [0. 1.]]

 [[1. 0.]
  [0. 1.]]

 [[1. 0.]
  [0. 1.]]

 [[1. 0.]
  [0. 1.]]

 [[1. 0.]
  [0. 1.]]

 [[1. 0.]
  [0. 1.]]

 [[1. 0.]
  [0. 1.]]

 [[1. 0.]
  [0. 1.]]

 [[1. 0.]
  [0. 1.]]]


In [33]:
# limit the number of labeled data to 1000
num_labeled=1000
index_train=tf.experimental.numpy.random.randint(0,x_train.shape[0],num_labeled)

x_train_labeled=x_train[index_train,:,:,:]
y_train_labeled=y_train[index_train,:]

In [52]:
import copy
x_train_fully=copy.deepcopy(x_train_labeled)
y_train_fully=copy.deepcopy(y_train_labeled)

x_train_downstream=copy.deepcopy(x_train_labeled)
y_train_downstream=copy.deepcopy(y_train_labeled)



In [45]:
# model creation

# Assuming x_train has the shape (batch_size, height, width, channels)
input_layer = tf.keras.Input(shape=(x_train.shape[1], x_train.shape[2], x_train.shape[3]))
reshaping=tf.keras.layers.Lambda(lambda x: tf.image.resize_with_pad(
    x,160,160, method=tf.image.ResizeMethod.BILINEAR))(input_layer)

model_DenseNet121 = tf.keras.applications.DenseNet121(include_top  = False,
                                                      weights      = None,
                                                      input_shape  = (160,160,3),
                                                      input_tensor = reshaping,
                                                      pooling      = 'max')
model_base_fully =  tf.keras.models.clone_model(model_DenseNet121)
model_base_pretext =  tf.keras.models.clone_model(model_DenseNet121)


model_base_fully.set_weights(model_DenseNet121.get_weights())
model_base_pretext.set_weights(model_DenseNet121.get_weights())

# normalization layers
layer_batchnorm_fsp = tf.keras.layers.BatchNormalization()
layer_batchnorm_prx = tf.keras.layers.BatchNormalization()


# output layers
print(f"y_train_fully: {y_train_fully.shape[-1]}")
output_layer_fully = tf.keras.layers.Dense(y_train_fully.shape[-1],
                                    activation = 'softmax')
print(f"x_train_pretext_output: {x_train_pretext_output.shape[-1]}")
output_layer_pretext = tf.keras.layers.Dense(x_train_pretext_output.shape[-1],
                                    activation = 'softmax')

y_train_fully: 10
x_train_pretext_output: 1


In [46]:
model_fsp   = tf.keras.models.Sequential([model_base_fully,
                                          layer_batchnorm_fsp,
                                          output_layer_fully])

model_prx   = tf.keras.models.Sequential([model_base_pretext,
                                          layer_batchnorm_prx,
                                          output_layer_pretext])

model_fsp.compile(optimizer = tf.keras.optimizers.Adam(0.001),
                  loss      = 'categorical_crossentropy',
                  metrics   = ['accuracy'])

model_prx.compile(optimizer = tf.keras.optimizers.Adam(0.001),
                  loss      = 'categorical_crossentropy',
                  metrics   = ['accuracy'])

In [47]:
print('Base model | fsp: ', model_fsp.layers[0].layers[10].weights[0][0][0][0][:5])
print('Base model | prx: ', model_prx.layers[0].layers[10].weights[0][0][0][0][:5])

print('Batch normalizaiton layer | fsp: ', model_fsp.layers[1].weights[:2])
print('Batch normalizaiton layer | prx: ', model_prx.layers[1].weights[:2])

'''
Let's keep model_fsp last layer's initial parameters for later use.
'''
layerou_fsp_initial_parameters = copy.deepcopy(model_fsp.layers[2].weights)

Base model | fsp:  tf.Tensor([-0.02986425  0.12511803 -0.17394464 -0.06576502 -0.05012904], shape=(5,), dtype=float32)
Base model | prx:  tf.Tensor([-0.02986425  0.12511803 -0.17394464 -0.06576502 -0.05012904], shape=(5,), dtype=float32)
Batch normalizaiton layer | fsp:  [<tf.Variable 'batch_normalization_2/gamma:0' shape=(1024,) dtype=float32, numpy=array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)>, <tf.Variable 'batch_normalization_2/beta:0' shape=(1024,) dtype=float32, numpy=array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)>]
Batch normalizaiton layer | prx:  [<tf.Variable 'batch_normalization_3/gamma:0' shape=(1024,) dtype=float32, numpy=array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)>, <tf.Variable 'batch_normalization_3/beta:0' shape=(1024,) dtype=float32, numpy=array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)>]


In [48]:
print(f"shape: {x_train_fully.shape}")
print(f"shape: {y_train_fully.shape}")

shape: (1000, 32, 32, 3)
shape: (1000, 10)


In [49]:
history_fsp = model_fsp.fit(x_train_fully,
                            y_train_fully,
                            epochs           = 5,
                            batch_size       = 32,
                            verbose          = 1,
                            shuffle          = True)

Epoch 1/5
32/32 [==============================] - 97s 421ms/step - loss: 2.3937 - accuracy: 0.2050
Epoch 2/5
32/32 [==============================] - 7s 215ms/step - loss: 1.8737 - accuracy: 0.3460
Epoch 3/5
32/32 [==============================] - 6s 193ms/step - loss: 1.6045 - accuracy: 0.4450
Epoch 4/5
32/32 [==============================] - 6s 197ms/step - loss: 1.3863 - accuracy: 0.5060
Epoch 5/5
32/32 [==============================] - 6s 191ms/step - loss: 1.1680 - accuracy: 0.6050


In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor              = 'val_loss',
                                            patience             = 2,
                                            restore_best_weights = True)

history_prx = model_prx.fit( x_train_pretext,
                            x_train_pretext_output,
                            epochs           = 5,
                            batch_size       = 32,
                            verbose          = 1,
                            shuffle          = True,
                            validation_split = 0.05,
                            callbacks        = [callback])

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


2969/2969 [==============================] - 642s 198ms/step - loss: 0.0000e+00 - accuracy: 0.5263 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
2969/2969 [==============================] - 578s 195ms/step - loss: 0.0000e+00 - accuracy: 0.5263 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/5
 595/2969 [=====>........................] - ETA: 7:37 - loss: 0.0000e+00 - accuracy: 0.5206

In [51]:
'''
Control if model_fsp has similar hyper-parameters as model_prx
'''

print('Base model | fsp: ', model_fsp.layers[0].layers[10].weights[0][0][0][0][:5])
print('Base model | prx: ', model_prx.layers[0].layers[10].weights[0][0][0][0][:5])

print('Batch normalizaiton layer | fsp: ', model_fsp.layers[1].weights[:2])
print('Batch normalizaiton layer | prx: ', model_prx.layers[1].weights[:2])

Base model | fsp:  tf.Tensor([-0.03393763  0.13035484 -0.18123257 -0.06270994 -0.05973897], shape=(5,), dtype=float32)
Base model | prx:  tf.Tensor([ 0.00906945  0.14211026 -0.16112804  0.03204448 -0.0425082 ], shape=(5,), dtype=float32)
Batch normalizaiton layer | fsp:  [<tf.Variable 'batch_normalization_2/gamma:0' shape=(1024,) dtype=float32, numpy=
array([0.997861  , 0.99324596, 0.9894802 , ..., 0.9934368 , 0.9850776 ,
       1.0035125 ], dtype=float32)>, <tf.Variable 'batch_normalization_2/beta:0' shape=(1024,) dtype=float32, numpy=
array([-0.00407274, -0.00872419, -0.00772908, ..., -0.01838654,
        0.0235607 , -0.0185999 ], dtype=float32)>]
Batch normalizaiton layer | prx:  [<tf.Variable 'batch_normalization_3/gamma:0' shape=(1024,) dtype=float32, numpy=
array([5.4645495, 5.4618316, 5.4398956, ..., 5.4236836, 5.4388657,
       5.4175067], dtype=float32)>, <tf.Variable 'batch_normalization_3/beta:0' shape=(1024,) dtype=float32, numpy=
array([ 4.323962 ,  4.298325 , -4.2573156, 

DownStreamTask

DOWNStream Task

In [53]:
layerou_dwm = tf.keras.layers.Dense(y_train_downstream.shape[-1],
                                    activation = 'softmax')

In [54]:
model_base_pretext.trainable      = False
layer_batchnorm_prx.trainable = False

model_dwm   = tf.keras.models.Sequential([model_base_pretext,
                                          layer_batchnorm_prx,
                                          layerou_dwm])

In [55]:
model_dwm.layers[2].set_weights(layerou_fsp_initial_parameters)


model_dwm.compile(optimizer = tf.keras.optimizers.Adam(0.001),
                  loss      = 'categorical_crossentropy',
                  metrics   = ['accuracy'])

In [56]:
model_base_pretext.trainable      = False
layer_batchnorm_prx.trainable = False

model_dwm   = tf.keras.models.Sequential([model_base_pretext,
                                          layer_batchnorm_prx,
                                          layerou_dwm])
model_dwm.summary()

history_dwm = model_dwm.fit(x_train_downstream ,
                             y_train_downstream,
                            epochs           = 2,
                            batch_size       = 35,
                            verbose          = 1,
                            shuffle          = True)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 1024)              7037504   
                                                                 
 batch_normalization_3 (Bat  (None, 1024)              4096      
 chNormalization)                                                
                                                                 
 dense_4 (Dense)             (None, 10)                10250     
                                                                 
Total params: 7051850 (26.90 MB)
Trainable params: 10250 (40.04 KB)
Non-trainable params: 7041600 (26.86 MB)
_________________________________________________________________
Epoch 1/2
29/29 [==============================] - 15s 168ms/step - loss: 3.2764 - accuracy: 0.1130
Epoch 2/2
29/29 [==============================] - 2s 57ms/step - loss: 2.5585 - accuracy: 0.1170


In [ ]:
model_base_pretext.trainable      = True
layer_batchnorm_prx.trainable = True

model_dwm   = tf.keras.models.Sequential([model_base_pretext,
                                          layer_batchnorm_prx,
                                          layerou_dwm])
model_dwm.summary()

history_dwm = model_dwm.fit(x_train_downstream ,
                             y_train_downstream,
                            epochs           = 2,
                            batch_size       = 35,
                            verbose          = 1,
                            shuffle          = True)

In [57]:
_, acc_te_fsp = model_fsp.evaluate(x_test,
                                    y_test,
                                   batch_size = 128)

_, acc_te_dwm = model_dwm.evaluate(x_test,
                                   y_test,
                                   batch_size = 128)

print('Accuracy of fsp: {:05.2f}%'.format(acc_te_fsp*100))
print('Accuracy of dwm: {:05.2f}%'.format(acc_te_dwm*100))

79/79 [==============================] - 17s 179ms/step - loss: 2.7575 - accuracy: 0.1320
Accuracy of fsp: 14.55%
Accuracy of dwm: 13.20%
